In [1]:
import numpy as np
from PIL import Image, ImageOps
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
from numba import vectorize

In [3]:
##############
## Subtasks ##
##############

def makefolder(path):
    try:
        os.mkdir(path)
    except:
        pass
def makesubfolder(path):
    try:
        os.makedirs(path)
    except:
        pass
    
def add_artifact(image,occult_ratio,aspect_ratio=1):
    mat_1 = np.array(image)[:,:,0]
    size,_ = mat_1.shape
    kernel_size = int(occult_ratio * size)
    px,py = [np.random.randint(0,high = size-kernel_size),np.random.randint(0,high = size-kernel_size)]

    artifact = np.zeros((kernel_size,kernel_size))
    mat_1[px:px + kernel_size , py:py + kernel_size] = artifact
    
    return mat_1

In [25]:
###############################
### Preprocessing Pipelines ###
###############################

def preprocessing_pipeline():
    size = np.linspace(100,300,5)
    occult_ratio = np.linspace(0.2,0.6,3)
    
    dir_1 = os.listdir('raw-img')
    dir_2 = ['raw-img/'+str(d) for d in dir_1]
    files = np.array([os.listdir(d) for d in dir_2])
    makefolder('groundtruth_output')
    makefolder('occulted_output')
    
    index = 0
    for s in size:
        desired_size = int(s)
        print(desired_size)
        dir_3 = 'groundtruth_output/resized_' + str(desired_size)
        makesubfolder(dir_3) 
        
        for i in range(len(files)):
            for j in tqdm(range(len(files[i])),position=0):
                
                im = Image.open(dir_2[i]+"/"+files[i][j])
                img_greyscaled = im.convert('LA')
                img_resized = img_greyscaled.resize((desired_size,desired_size))    
                img_resized.save(dir_3 + '/' + files[i][j], "PNG")
                
                for r in occult_ratio:
                    path = 'occulted_output/'+ str(int(100*r)) + '_percent/' + str(int(size[index]))
                    makesubfolder(path)
                    test = img_resized
                    img_occulted = add_artifact(test,r)
                    img_final = Image.fromarray(img_occulted)
                    img_final.save(path +'/'+ files[i][j],"PNG")
        index += 1
        
preprocessing_pipeline()